In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import skimage as ski
from skimage.morphology import disk

# ETHZ: 227-0966-00L
# Quantitative Big Imaging
# February 21, 2019

## Image Enhancement

# Measurements are rarely perfect
<figure><img src="ext-figures/lecture03/imperfect_imaging_system.svg" style="height:300px" align="middle"></figure>

## Factors affecting the image quality
* Resolution (Imaging system transfer functions)
* Noise
* Contrast
* Inhomogeneous contrast
* Artifacts

No measurement system is perfect and therefore, you will also not get perfect images of the sample you measured. In the figure you can see the slice as it would be in a perfect world to the left and what you actually would measure with an imaging system. Some are of higher quality than others but there are still a collection of factors that have an impact on the image quality.
* __Resolution__ Thanks to the optical system there will be some unsharpness caused by scintillator, lenses, and camera. this appears as blurring or the acquired images.
* __Noise__ There are different sources of noise when an image is acquired. Most of them are Poisson/Binomial distributed point processes, but thanks to the light spread in the scintillator there can also be some spatially correlated noise.
* __Contrast__ Sometimes there is a very low intensity difference between different regions of the sample. This is in particularly problematic when the signal to noise ratio is low.
* __Inhomogeneous contrast__ The interaction between radiation and matter may introduce effects that appear as an inhomogeneous contrast. Typical such effects are beam hardening and scattering.
* __Artifacts__ The artifacts are features in the image that are not present in the sample. Often this is caused by detector errors or interference of radiation background or seconday radiation events.  


# A typical processing chain

<figure><img src="ext-figures/lecture03/image_proc_chain.svg" style="height:300px" align="middle"></figure>
Today's lecture will focus on the enhancement

Traditionally, the processing of image data can be divided into a series of sub tasks that provide the final resul. 
* __Acquisition__ The data must obviously be acquired and stored. There are cases when simulated data is used. Then, the acquisition is replaced by the process to simulate the data.
* __Enhancement__ The raw data is usually not ready to be processed in the form is comes from the acquisition. It usually has noise and artifacts as we saw on the previous slide. The enhancement step suppresses unwanted information in the data.
* __Segmentation__ The segmenation identifies different regions based on different features such as intensity distribution and shape.
* __Post processing__ After segmentation, there may be falsely identified regions. These are removed in a post processing step. 
* __Evaluation__ The last step of the process is to make conclusions based on the image data. It could be modelling material distrbutions, measuring shapes etc.

# Noise and artifacts
## The unwanted information

# Noise types
* Spatially uncorrelated noise
* Event noise
* Stuctured noise

### Noise examples

In [ ]:
plt.figure(figsize=[12,5]);
plt.subplot(1,3,1);plt.imshow(np.random.normal(0,1,[100,100])); plt.title('Gaussian');plt.axis('off');
plt.subplot(1,3,2);plt.imshow(0.90<np.random.uniform(0,1,size=[100,100]),cmap='gray'); plt.title("Salt and pepper"),plt.axis('off');
plt.subplot(1,3,3);plt.imshow(ski.filters.gaussian(np.random.normal(0,1,size=[100,100]),sigma=1),cmap='gray'); plt.title("Structured"),plt.axis('off');

# Gaussian noise
* Additive
* Easy to model 
* Law of large numbers

### Distribution function
$n(x)=\frac{1}{\sqrt{2\pi\sigma}}\exp{-\left(\frac{x-\mu}{2\sigma}\right)^2}$

# Poisson noise
* Multiplicative
* Physically correct for event counting

### Distribition function 
$p(x)=\frac{\lambda^{k}}{k!} \exp{-\lambda\,x}$

# Compare Gaussian and Possion noise

In [ ]:
plt.figure(figsize=[15,8])
x=np.linspace(0,2*np.pi,100); 
y=10*np.sin(x)+11; ng=np.random.normal(0,1,size=len(x)); npoi = np.random.poisson(y);
plt.subplot(2,2,1); plt.plot(x,y+ng);plt.plot(x,y); plt.axis('off');plt.title('Gaussian'); plt.subplot(2,2,3);plt.plot(x,ng);plt.axis('off');
plt.subplot(2,2,2); plt.plot(x,npoi);plt.plot(x,y); plt.axis('off');plt.title('Poisson'); plt.subplot(2,2,4);plt.plot(x,npoi-y);plt.axis('off');

# Noise models - Salt'n'pepper noise
* A type of outlier noise
* Noise frequency described as probability of outlier
* Can be additive, multiplicative, and independent replacement

### Example model
$sp(x)=\left\{\begin{array}{ll}
-1 & x\leq\lambda_1\\ 
0 & \lambda_1< x \leq \lambda_2\\
1 & \lambda_2<x
\end{array}\right.\qquad \begin{array}{l}x\in\mathcal{U}(0,1)\\\lambda_1<\lambda_2\\
\lambda_1+\lambda_2 = \mbox{noise fraction}
\end{array}$

# Salt'n'pepper examples

In [ ]:
def snp(dims,Pblack,Pwhite) : # Noise model function
    uni=np.random.uniform(0,1,dims)
    img=(Pwhite<uni).astype(float)-(uni<Pblack).astype(float)
    return img

In [ ]:
img10_90=snp([100,100],0.1,0.9); img5_95=snp([100,100],0.05,0.95);img1_99=snp([100,100],0.01,0.99)
plt.figure(figsize=[15,5])
plt.subplot(1,3,1); plt.imshow(img1_99,cmap='gray'); plt.title('$\lambda_1$=1% and $\lambda_2$=99%',fontsize=16); plt.axis('off');
plt.subplot(1,3,2); plt.imshow(img5_95,cmap='gray'); plt.title('$\lambda_1$=5% and $\lambda_2$=95%',fontsize=16); plt.axis('off');
plt.subplot(1,3,3); plt.imshow(img10_90,cmap='gray'); plt.title('$\lambda_1$=10% and $\lambda_2$=90%',fontsize=16); plt.axis('off');

# Signal to noise ratio

The Signal to noise ratio measures the noise strengh in a signal

### Definition

$SNR=\frac{mean(f)}{stddev(f)}$

# Signal to noise ratio for Poisson noise
- For a Poisson distribution the $SNR \sim \sqrt{N}$ 
- $N$ is the number of particles $\sim$ exposure time

In [ ]:
exptime=np.array([50,100,200,500,1000,2000,5000,10000])
snr = np.array([ 8.45949767, 11.40011621, 16.38118766, 21.12056507, 31.09116641,40.65323123, 55.60833117, 68.21108979]);
marker_style = dict(color='cornflowerblue', linestyle='-', marker='o',markersize=10, markerfacecoloralt='gray');
plt.figure(figsize=(15,6)) 
plt.subplot(1,3,2);plt.plot(exptime/1000,snr, **marker_style);plt.xlabel('Exposure time [s]');plt.ylabel('SNR [1]')
img50ms=plt.imread('ext-figures/lecture03/tower_50ms.png'); img10000ms=plt.imread('ext-figures/lecture03/tower_10000ms.png');
plt.subplot(1,3,1);plt.imshow(img50ms); plt.subplot(1,3,3); plt.imshow(img10000ms);

# Useful python functions

### Random number generators [numpy.random]
Generate an $m \times n$ random fields with different distributions:
* __Gauss__ ```np.random.normal(mu,sigma, size=[rows,cols])```
* __Uniform__ ```np.random.uniform(low,high,size=[rows,cols])```
* __Poisson__ ```np.random.poisson(lambda, size=[rows,cols])
	
### Statistics

* ```np.mean(f)```, ```np.var(f)```, ```np.std(f)``` Computes the mean, variance, and standard deviation of an image $f$.
* ```np.min(f)```,```np.max(f)``` Finds minimum and maximum values in $f$.
* ```np.median(f)```, ```np.rank()``` Selects different values from the sorted data.


# Basic filtering

# What is a filter?

### General definition
A filter is a processing unit that
* Enhances the wanted information 
* Suppresses the unwanted information

<div class="alert alert-block alert-warning">
Ideally without altering relevant features beyond recognition .
</div>



# Filter characteristics

Filters are characterized by the type of information they suppress

# Low-pass filters
* Slow changes are enhanced 
* Rapid changes are suppressed 
<figure><img src="ext-figures/lecture03/lp_principle.svg" style="height:300px" align="middle"></figure>

# High-pass filters
* Rapid changes are enhanced 
* Slow changes are suppressed
<figure><img src="ext-figures/lecture03/hp_principle.svg" style="height:300px" align="middle"></figure>

# Basic filters

## Linear filters
Computed using the convolution operation

$$g(x)=h*f(x)=\int_{\Omega}f(x-\tau) h(\tau) d\tau$$
where

* __$f$__ is the image 
* __$h$__ is the convolution kernel of the filter

<figure><img src="ext-figures/lecture03/filter_box.svg" style="height:150px" align="middle"></figure>

## Low-pass filter kernels
Low-pass filters suppress noise

### Mean or Box filter
All weights have the same value.

Example:
$$B=\frac{1}{25}\cdot\begin{array}{|c|c|c|c|c|}
\hline
1 & 1 & 1 & 1& 1\\
\hline
1 & 1 & 1 & 1& 1\\
\hline
1 & 1 & 1 & 1& 1\\
\hline
1 & 1 & 1 & 1& 1\\
\hline
1 & 1 & 1 & 1& 1\\
\hline
\end{array}
$$

### Gauss filter
$$G=\exp{-\frac{x^2+y^2}{2\,\sigma^2}}$$
Example:
<figure><img src="ext-figures/lecture03/gauss_bell.svg" style="height:300px" align="middle"></figure>

# Using a Gauss filter

In [ ]:
img = plt.imread('ext-figures/lecture03/input_orig.png'); 
noise = np.random.normal(0,1,size=img.shape); snr10=img+0.1*noise; snr5=img+0.2*noise; snr2=img+0.5*noise;plt.figure(figsize=[15,8])
plt.subplot(3,4,1); plt.imshow(img); plt.axis('off');plt.title('No noise'); plt.subplot(3,4,2); plt.imshow(snr10); plt.title('SNR=10'); plt.axis('off');plt.subplot(3,4,3); plt.imshow(snr5); plt.title('SNR=5'); plt.axis('off');plt.subplot(3,4,4); plt.imshow(snr2); plt.title('SNR=2'); plt.axis('off');
plt.subplot(3,4,5); plt.imshow(ski.filters.gaussian(img,sigma=1)); plt.title('Gaussian $\sigma$=1'); plt.axis('off');plt.subplot(3,4,6); plt.imshow(ski.filters.gaussian(snr10,sigma=1));  plt.axis('off');plt.subplot(3,4,7); plt.imshow(ski.filters.gaussian(snr5,sigma=1));  plt.axis('off');plt.subplot(3,4,8); plt.imshow(ski.filters.gaussian(snr2,sigma=1)); plt.axis('off');
plt.subplot(3,4,9); plt.imshow(ski.filters.gaussian(img,sigma=3)); plt.title('Gaussian $\sigma$=3'); plt.axis('off');plt.subplot(3,4,10); plt.imshow(ski.filters.gaussian(snr10,sigma=3));  plt.axis('off');plt.subplot(3,4,11); plt.imshow(ski.filters.gaussian(snr5,sigma=3));  plt.axis('off');plt.subplot(3,4,12); plt.imshow(ski.filters.gaussian(snr2,sigma=3)); plt.axis('off');

## How is the convolution computed

<figure><img src="ext-figures/lecture03/principle_mean_filter.svg" style="height:400px" align="middle"></figure>

<div class="alert alert-block alert-success">
For a non-uniform kernel each term is weighted by its kernel weight.
</div>

## Euclidean separability
The asociative and commutative laws apply to convoution

$$(a * b)*c=a*(b*c) \quad \mbox{ and } \quad a * b = b * a $$

A convolution kernel is called _separable_ if it can be split in two or more parts:
<font size="3">
$$\begin{array}{|c|c|c|}
\hline
\cdot & \cdot & \cdot\\
\hline
\cdot & \cdot & \cdot\\
\hline
\cdot & \cdot& \cdot\\
\hline
\end{array}=
\begin{array}{|c|}
\hline
\cdot \\
\hline
\cdot \\
\hline
\cdot \\
\hline
\end{array}
*
\begin{array}{|c|c|c|}
\hline
\cdot & \cdot & \cdot\\
\hline
\end{array}$$
</font>    

$$\exp{-\frac{x^2+y^2}{2\,\sigma^2}}=\exp{-\frac{x^2}{2\,\sigma^2}}*\exp{-\frac{y^2}{2\,\sigma^2}}$$

#### Gain
Separability reduces the number of computations $\rightarrow$ faster processing
- 3$\times$3 $\rightarrow$ 9 mult and 8 add $\Leftrightarrow$ 6 mult and 4 add
- 3$\times$3$\times$3 $\rightarrow$ 27 mult and 26 add $\Leftrightarrow$ 9 mult and 6 add


## The median filter
<figure><img src="ext-figures/lecture03/principle_median_filter.svg" style="height:400px" align="middle"></figure>

## Comparing filters for different noise types

In [ ]:
img = plt.imread('ext-figures/lecture03/grasshopper.png'); noise=img+np.random.normal(0,0.1,size=img.shape); spots=img+0.2*snp(img.shape,0,0.8); noise=(noise-noise.min())/(noise.max()-noise.min());spots=(spots-spots.min())/(spots.max()-spots.min())
plt.figure(figsize=[15,7])
plt.subplot(2,3,1); plt.imshow(spots); plt.title('Spots'); plt.axis('off');
plt.subplot(2,3,2); plt.imshow(ski.filters.gaussian(spots,sigma=1)); plt.title('Gauss filter'); plt.axis('off');
plt.subplot(2,3,3); plt.imshow(ski.filters.median(spots,disk(3))); plt.title('Median'); plt.axis('off');

plt.subplot(2,3,4); plt.imshow(noise); plt.title('Gaussian noise'); plt.axis('off');
plt.subplot(2,3,5); plt.imshow(ski.filters.gaussian(noise,sigma=1)); plt.title('Gauss filter'); plt.axis('off');
plt.subplot(2,3,6); plt.imshow(ski.filters.median(noise,disk(3))); plt.title('Median'); plt.axis('off');

## Filter example: Spot cleaning

### Problem
- Many neutron images are corrupted by spots that confuse following processing steps. 
- The amount, size, and intensity varies with many factors. 
### Solutions
- :-( Low pass filter
- :-( Median filter
- :-) Detect spots and replace by estimate

### Example
<figure><img src="ext-figures/lecture03/spotty_knot_closeup.png" style="height:400px" align="middle"></figure>

## Spot cleaning algorithm

<figure><img src="ext-figures/lecture03/spotclean_algorithm.svg" style="height:400px" align="middle"></figure>

#### Parameters

- $N$ Width of median filter.
- $k$ Threshold level for outlier detection.


## Spot cleaning - Compare performance
<figure><img src="ext-figures/lecture03/spotclean_compare.pdf"  style="height:500px" align="middle"></figure>

### The ImageJ ways

- __Despeckle__ Median ... please avoid this one!!!
- __Remove outliers__ Similar to cleaning described algorithm

## High-pass filters
High-pass filters enhance rapid changes $\rightarrow$ ideal for edge detection

### Typical high-pass filters:

#### Gradients
$$\frac{\partial}{\partial\,x}=\frac{1}{2}\cdot\begin{array}{|c|c|}
\hline
-1 & 1\\
\hline
\end{array}\qquad
\frac{\partial}{\partial\,x}=\frac{1}{32}\cdot\begin{array}{|c|c|c|}
\hline
-3 & 0 & 3\\
\hline
-10 & 0 & 10\\
\hline
-3 & 0 & 3\\
\hline
\end{array}
$$
#### Laplacian
$$
\bigtriangleup=\frac{1}{2}\cdot\begin{array}{|c|c|c|}
\hline
1 & 2 & 1\\
\hline
2 & -12 & 2\\
\hline
1 & 2 & 1\\
\hline
\end{array}
$$
#### Sobel
$$
G=|\nabla f|=\sqrt{\left(\frac{\partial}{\partial\,x}f\right)^2 + \left(\frac{\partial}{\partial\,y}f\right)^2}
$$


## Gradient example
<img src="ext-figures/lecture03/orig.png" style="height:150px">

#### Vertical edges
$\frac{\partial}{\partial x}=\frac{1}{32}\cdot\begin{array}{|c|c|c|}
\hline
-3 & 0 & 3\\
\hline
-10 & 0 & 10\\
\hline
-3 & 0 & 3\\
\hline
\end{array} \ast$

#### Horizontal egdges
$\frac{\partial}{\partial\,y}=\frac{1}{32}\cdot\begin{array}{|c|c|c|}
\hline
-3 & -10 & -3\\
\hline
0 & 0 & 0\\
\hline
3 & 10 & 3\\
\hline
\end{array}\,\ast\, =$

[Jaehne, 2005](https://doi.org/10.1007/3-540-27563-0)

## Edge detection examples

In [ ]:
img=plt.imread('ext-figures/lecture03/orig.png');
plt.figure(figsize=[15,6])
plt.subplot(1,2,1);plt.imshow(ski.filters.laplace(img),clim=[-0.1,0.1]); plt.title('Laplacian'); plt.colorbar();
plt.subplot(1,2,2);plt.imshow(ski.filters.sobel(img)); plt.title('Sobel');  plt.colorbar();